# Download and store prebuilt models.

In [1]:
import os
from urllib.request import urlopen
from urllib.parse import urlparse

In [2]:
download_dir = os.path.join("var", "downloads")
model_dir = os.path.join("models")

model_files = [
    # YOLOv8 Nano ONNX
    "https://huggingface.co/SpotLab/YOLOv8Detection/resolve/3005c6751fb19cdeb6b10c066185908faf66a097/yolov8n.onnx?download=true",
    
    # YOLOv4-tiny files (legacy)
    "https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov4-tiny.weights",
    "https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4-tiny.cfg",
    
    # COCO class names (shared by both models)
    "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names"
]

In [3]:
def download(url, dest_file, timeout=10):
    tmp_file = dest_file + ".part"
    with urlopen(url, timeout=timeout) as resp:
        if resp.status != 200:
            raise RuntimeError(f"HTTP {resp.status} for {url}")

        try:
            with open(tmp_file, "wb") as f:
                for chunk in iter(lambda: resp.read(1024 * 8), b""):
                    f.write(chunk)

            os.replace(tmp_file, dest_file)  # atomic on POSIX + Windows
        except Exception:
            os.unlink(tmp_file)
            raise

def download_url_cached(url):
    os.makedirs(download_dir, exist_ok=True)

    path = urlparse(url).path
    file_name = os.path.basename(path)
    download_file = os.path.join(download_dir, file_name)

    if os.path.isfile(download_file):
        print(f"Using cached file {download_file}")
        return download_file
    
    download(url, download_file)
    return download_file

def download_install(url):
    download_file = download_url_cached(url)
    os.makedirs(model_dir, exist_ok=True)
    dest_file = os.path.join(model_dir, os.path.basename(download_file))
    if not os.path.isfile(dest_file):
        print(f"Installing model file to {dest_file}")
        os.link(download_file, dest_file)
    else:
        print(f"Model file already installed at {dest_file}")

In [4]:
for url in model_files:
    print(f"Processing URL: {url}")
    download_install(url)

Processing URL: https://huggingface.co/SpotLab/YOLOv8Detection/resolve/3005c6751fb19cdeb6b10c066185908faf66a097/yolov8n.onnx?download=true
Using cached file var/downloads/yolov8n.onnx
Installing model file to models/yolov8n.onnx
Processing URL: https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov4-tiny.weights
Using cached file var/downloads/yolov4-tiny.weights
Installing model file to models/yolov4-tiny.weights
Processing URL: https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4-tiny.cfg
Using cached file var/downloads/yolov4-tiny.cfg
Installing model file to models/yolov4-tiny.cfg
Processing URL: https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names
Using cached file var/downloads/coco.names
Installing model file to models/coco.names
